In [3]:
%pip install fastparquet

   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   --------------- ------------------------ 262.1/673.3 kB ? eta -:--:--
   ---------------------------------------- 673.3/673.3 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 1.8 MB/s eta 0:00:01
   -------------------- ------------------- 1.0/2.1 MB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.1 MB 1.6 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np 


In [4]:
df = pd.read_parquet('./assignment6_data.parquet')


In [5]:
df.head()

,Published_Sample_Name,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,RP5-890E16.5,RP11-122L4.2,LINC00624,RP5-1116H23.4,RP5-1099C19.2,RP3-405J10.5,MRPL45,RP11-327I22.5,SubGroup,BA
0,20131219_1C01,11.667,0.0,2.083,0.208,0.417,0.000,0.0,0.0,508.138000,...,0.000,0.000,0.833,0.0,0.625,0.625,1.042,0.0,Ex1,BA8
1,20131219_1C07,0.000,0.0,2.228,27.541,5.468,0.000,0.0,0.0,6.480000,...,0.810,0.000,0.608,0.0,0.608,0.000,8.505,0.0,Ex3,BA8
2,20131219_1C09,0.000,0.0,69.514,0.000,1.209,0.000,0.0,0.0,7.254000,...,1.813,1.511,0.907,0.0,0.000,0.000,4.533,0.0,Ex4,BA8
3,20131219_1C13,0.000,0.0,3.956,1.130,0.424,0.141,0.0,0.0,114.302002,...,0.565,0.141,0.141,0.0,0.000,0.000,1.554,0.0,Ex1,BA8
4,20131219_1C30,0.000,0.0,3.170,0.059,10.155,0.117,0.0,0.0,2.113000,...,0.000,0.000,1.702,0.0,0.176,0.000,0.470,0.0,Ex3,BA8


In [6]:
df.columns.value_counts()

BA                       1
Published_Sample_Name    1
RP11-706P11.2            1
RP11-694O4.1             1
RP11-227G15.11           1
                        ..
FGR                      1
C1orf112                 1
SCYL3                    1
DPM1                     1
TNMD                     1
Name: count, Length: 25010, dtype: int64

In [7]:
df.isnull().sum()

Published_Sample_Name    0
TSPAN6                   0
TNMD                     0
DPM1                     0
SCYL3                    0
                        ..
RP3-405J10.5             0
MRPL45                   0
RP11-327I22.5            0
SubGroup                 0
BA                       0
Length: 25010, dtype: int64

In [11]:
# for cols in df.columns:
#     print(df[cols].unique())

In [ ]:
categorical_columns = df.select_dtypes(include='object').columns

if len(categorical_columns) > 0:
    print("Categorical columns present:", list(categorical_columns))
else:
    print("No categorical columns found.")

Categorical columns present: ['Published_Sample_Name', 'SubGroup', 'BA']


In [12]:
variances = df.select_dtypes(include='number').var()
sorted_variances_list = variances.sort_values(ascending=False).tolist()
print(sorted_variances_list)

[155851808.0, 93079392.0, 59179552.0, 52402452.0, 31620300.0, 19143158.0, 18212508.0, 17316110.0, 15338256.0, 12415458.0, 9749509.0, 8812264.0, 7444455.0, 7190681.0, 6633897.5, 6182442.0, 6150984.5, 5983215.5, 5365486.5, 4904321.5, 4878254.0, 4774800.5, 4462694.5, 4456769.5, 4456108.5, 3874755.5, 3868290.0, 3739473.5, 3188729.25, 3028212.0, 3004026.75, 2920962.75, 2891940.25, 2733538.0, 2729484.5, 2603452.5, 2542864.5, 2533174.25, 2359213.25, 2269486.75, 2239241.0, 2202231.75, 2136283.25, 2120760.5, 2109567.75, 2076072.0, 1993376.125, 1882819.75, 1818687.25, 1806927.125, 1794639.75, 1775277.125, 1735466.75, 1713713.75, 1592317.375, 1575440.125, 1551862.375, 1531626.875, 1515168.75, 1509372.5, 1486547.125, 1471530.75, 1469977.0, 1426656.125, 1393314.25, 1325514.0, 1303755.125, 1284307.875, 1260654.625, 1258909.875, 1218964.875, 1203634.0, 1179822.375, 1157834.0, 1147794.625, 1137440.25, 1120391.375, 1108944.125, 1100028.125, 1093991.625, 1086211.5, 1082040.625, 1065647.5, 1040543.5, 103